In [ ]:
from zomathon import ZomatoAPI
import pandas as pd
import folium
import jovian

In [ ]:
zom = ZomatoAPI('3943f8d142f7132a1c68e94b872649b2') #enter your API key

name = [] #store the name 
location=[]  #Store the location 
lat=[]  #store the latitude
lng=[]   #store the longitude
avg_cost_2=[]  #store the average cost
price_rnge=[]  #store the price range
rating=[]    #store the rating

for x in range(0,100,20):
    res = zom.search(city_id=259,start=x,count=20,cuisines=148,category=10)  
    for a in res['restaurants']:
        name.append(a['restaurant']['name'])
        location.append(a['restaurant']['location']['address'])
        lat.append(a['restaurant']['location']['latitude'])
        lng.append(a['restaurant']['location']['longitude'])
        avg_cost_2.append(a['restaurant']['average_cost_for_two'])
        price_rnge.append(a['restaurant']['price_range'])
        rating.append(a['restaurant']['user_rating']['aggregate_rating'])
    
col = {'Name':name,'Location':location,'Latitude':lat,'Longitude':lng,'Average cost for 2':avg_cost_2,'Price range':price_rnge,'Rating':rating}    
df= pd.DataFrame(col)    

In [ ]:
df

In [ ]:
df.sort_values('Rating', ascending=False)

In [ ]:
#calculating mean
df["Average cost for 2"].mean()

In [ ]:
#removing outliers and calculating mean
df[df['Average cost for 2']<=100]['Average cost for 2'].mean()

In [ ]:
df.groupby('Average cost for 2')['Average cost for 2'].count().plot.bar()

In [ ]:
#plotting the data
fig, axs = plt.subplots(1,2,figsize=(18,10))
df.hist('Average cost for 2',ax=axs[0])
df.boxplot(column = 'Average cost for 2',ax=axs[1])

In [ ]:
mel_map = folium.Map([-37.840935, 144.946457], zoom_start=10)
mel_map

In [ ]:
for index, row in df.iterrows():
    if row['Average cost for 2']<=59.8:
        folium.Marker([row['Latitude'], row['Longitude']],
                      popup=row['Name']+","+" "+"Rating="+str(row['Rating']),
                      tooltip=row['Location'],
                      icon=folium.Icon(color='green')
                     ).add_to(mel_map)
    else:
        folium.Marker([row['Latitude'], row['Longitude']],
                      popup=row['Name']+","+" "+"Rating="+str(row['Rating']),
                      tooltip=row['Location'],
                      icon=folium.Icon(color='red')
                     ).add_to(mel_map)
      
mel_map    

In [ ]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6378 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [ ]:
#latitude and longitude for the centre of CBD
cbd_lat,cbd_lng =  -37.81239161656876, 144.96227284082644

In [ ]:
#calculating haversine distance and storing in a new column
for index,row in df.iterrows():
    res_lat = float(row['Latitude'])
    res_lng = float(row['Longitude'])
    distance_from_cbd = haversine(cbd_lng,cbd_lat,res_lng,res_lat)
    df.loc[index,'Distance from CBD']=distance_from_cbd

In [ ]:
df.sort_values('Distance from CBD', ascending=False)

In [ ]:
cbd_res = df[df['Distance from CBD']<1]

In [ ]:
cbd_res

In [ ]:
cbd_res["Average cost for 2"].mean()

In [ ]:
#plotting data 
fig, axs = plt.subplots(1,2,figsize=(18,10))
cbd_res.hist('Average cost for 2',ax=axs[0])
cbd_res.boxplot(column = 'Average cost for 2',ax=axs[1])

In [ ]:
#calculating median
cbd_res["Average cost for 2"].median()

In [ ]:
cbd_map = folium.Map([-37.840935, 144.946457], zoom_start=10)
cbd_map

In [ ]:
for index, row in cbd_res.iterrows():
    if row['Average cost for 2']<=50:
        folium.Marker([row['Latitude'], row['Longitude']],
                      popup=row['Name']+","+" "+"Rating="+str(row['Rating']),
                      tooltip=row['Location'],
                      icon=folium.Icon(color='green')
                     ).add_to(cbd_map)
    else:
        folium.Marker([row['Latitude'], row['Longitude']],
                      popup=row['Name']+","+" "+"Rating="+str(row['Rating']),
                      tooltip=row['Location'],
                      icon=folium.Icon(color='red')
                     ).add_to(cbd_map)
cbd_map        

In [ ]:
out_res = df[df['Distance from CBD']>20]

In [ ]:
out_res

In [ ]:
#calculating mean
out_res["Average cost for 2"].mean()

In [ ]:
#plotting data
fig, axs = plt.subplots(1,2,figsize=(18,10))
out_res.hist('Average cost for 2',ax=axs[0])
out_res.boxplot(column = 'Average cost for 2',ax=axs[1])

In [ ]:
out_map = folium.Map([-37.840935, 144.946457], zoom_start=10)
out_map

In [ ]:
for index, row in out_res.iterrows():
    if row['Average cost for 2']<=53.1:
        folium.Marker([row['Latitude'], row['Longitude']],
                      popup=row['Name']+","+" "+"Rating="+str(row['Rating']),
                      tooltip=row['Location'],
                      icon=folium.Icon(color='green')
                     ).add_to(out_map)
    else:
        folium.Marker([row['Latitude'], row['Longitude']],
                      popup=row['Name']+","+" "+"Rating="+str(row['Rating']),
                      tooltip=row['Location'],
                      icon=folium.Icon(color='red')
                     ).add_to(out_map)

out_map